In [ ]:
from urllib.error import URLError
from urllib.request import urlopen
import re
import pymysql
import ssl
from pymysql import Error

def decode_page(page_bytes, charsets=('utf-8',)):
    page_html = None
    for charset in charsets:
        try:
            page_html = page_bytes.decode(charset)
            break
        except UnicodeDecodeError:
            print('UnicodeDecodeError')
            pass
    return page_html


def get_page_html(seed_url, *, retry_times=3, charsets=('utf-8',)):
    page_html = None
    try:
        page_html = decode_page(urlopen(seed_url).read(), charsets)
    except URLError:
        if retry_times > 0:
            return get_page_html(seed_url, retry_times=retry_times - 1, charsets=charsets)
    except:
        print('aooooo')
    return page_html

def get_matched_parts(page_html, pattern_str, pattern_ignore_case=re.I):
    pattern_regex = re.compile(pattern_str, pattern_ignore_case)
    return pattern_regex.findall(page_html) if page_html else []

def start_crawl(seed_url, match_pattern, *, max_depth=-1):
    conn = pymysql.connect(
        host='127.0.0.1', 
        port=3306, 
        database='d1', 
        charset='utf8', 
        user='root', 
        password='2313'
    )
    try:
        with conn.cursor() as cursor:
            url_list = [seed_url]
            visited_url_list = {seed_url: 0}
            while url_list:
                current_url = url_list.pop(0)
                depth = visited_url_list[current_url]
                if depth != max_depth:
                    # 用'utf-8', 'gbk', 'gb2312'三种字符集解码网页
                    page_html = get_page_html(current_url, charsets=('utf-8', 'gbk', 'gb2312'))
#                     print(page_html)
                    # 匹配出网页中的链接
#                     links_list = get_matched_parts(page_html, match_pattern)
#                     param_list = []
#                     for link in links_list:
#                         if link not in visited_url_list:
#                             visited_url_list[link] = depth + 1
#                             page_html = get_page_html(link, charsets=('utf-8', 'gbk', 'gb2312'))
#                             headings = get_matched_parts(page_html, r'<h1>(.*)<span')
#                             if headings:
#                                 param_list.append((headings[0], link))
#                     cursor.executemany('insert into tb_result (title, link) values (%s, %s)', params_list)
#                     conn.commit()
    except Error:
        print('Error')
        pass
    finally:
        conn.close()

def main():
    ssl._create_default_https_context = ssl._create_unverified_context
    start_crawl('http://sports.sohu.com.nba_a.shtml', r'<a[^>]+test=a\s[^>]*href=["\'](.*?)["\']', max_depth=2)

if __name__ == '__main__':
    main()


In [ ]:
drop database if exists d1;
create database d1 default charset utf8;

use tb_result;

drop table if exists tb_result;

create table tb_result
(
id   int not null auto_increment comment 'id',
title varchar(50) not null comment '标题',
link  varchar(50) not null comment '链接',
primary key (id)
);
